In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
period_data = []
period_data_pct = []
period_data_pct_m = []

period_rate = []
period_msci = []

In [3]:
data = pd.read_excel('./data/data.xlsx')
data['Unnamed: 0'] = pd.to_datetime(data['Unnamed: 0'])
data = data.set_index('Unnamed: 0')
data.index.name = None

#data slicer
indmin = data.groupby(data.index.to_period('M')).apply(lambda x: x.index.min()).values
indmax = data.groupby(data.index.to_period('M')).apply(lambda x: x.index.max()).values

for i in range(0, 61):
    pre = data.loc[indmin[i]:indmax[i+11]]
    
    if len(pre) == 366:
        pre = pre[1:]
    period_data.append(pre)
    
    period_data_pct.append(pre.pct_change().dropna())
    
pre = data.groupby([data.index.year, data.index.month]).agg(["first", "last"])

data_pct_m = pd.DataFrame(index = data.groupby(data.index.to_period('M')).apply(lambda x: x.index.max()).values
                     , columns = data.columns)
for i in data.columns:
    data_pct_m[i] = ( (pre[i]['last'] - pre[i]['first']) / pre[i]['first'] ).values

for i in range(0, 61):
    period_data_pct_m.append(data_pct_m.loc[indmin[i]:indmax[i+11]])

In [4]:
rate = pd.read_csv('./data/rates.csv', sep=',')
rate = rate[['Дата', 'Цена']]
rate[['Day', 'Month', 'Year']] = rate['Дата'].str.split('.',expand=True)
rate['Date'] = rate['Day'] + '-' + rate['Month'] + '-' + rate['Year']
rate = rate.drop(['Дата', 'Day', 'Month', 'Year'], axis=1)
rate['Date'] = pd.to_datetime(rate['Date'], format='%d-%m-%Y')
rate = rate.set_index('Date')
rate.index.name = None
rate['Цена'] = rate['Цена'].str.replace(',', '.')
rate['Цена'] = rate['Цена'].astype(float) / 100
rate.rename(columns={'Цена': 'Rate'}, inplace=True)
rate = rate.sort_index(ascending=True)

for i in range(0, 61):
    period_rate.append(rate.iloc[11+i, 0])

In [5]:
msci = pd.read_csv('./data/MSCI.csv', sep=',')
msci = msci[['Дата', 'Цена']]
msci[['Day', 'Month', 'Year']] = msci['Дата'].str.split('.',expand=True)
msci['Date'] = msci['Day'] + '-' + msci['Month'] + '-' + msci['Year']
msci = msci.drop(['Дата', 'Day', 'Month', 'Year'], axis=1)
msci['Date'] = pd.to_datetime(msci['Date'], format='%d-%m-%Y')
msci = msci.set_index('Date')
msci.index.name = None
msci['Цена'] = msci['Цена'].str.replace('.', '')
msci['Цена'] = msci['Цена'].str.replace(',', '.')
msci['Цена'] = msci['Цена'].astype(float)
msci.rename(columns={'Цена': 'MSCI'}, inplace=True)
msci = msci.sort_index(ascending=True)

for i in range(0, 61):
    period_msci.append(msci.loc[indmin[i]:indmax[i+11]])

Portfolio functions

In [6]:
from pypfopt import EfficientFrontier

In [7]:
def get_ef_dots(mu, cov_m):
    ef = EfficientFrontier(mu, cov_m, solver='SCS')
    mu_min = min(i for i in mu if i > 0)
    target_returns = np.linspace(mu_min*1.01, mu.max()*0.99, 100)
    sigmas = []
    weights = []
    for target_return in target_returns:
        ef.efficient_return(target_return)
        weight = ef.clean_weights()
        weights.append(list(weight.values()))
        sigmas.append(ef.portfolio_performance()[1])
    cut = list(map(lambda i: i > np.diag(cov_m).min(), sigmas)).index(True)
    return [target_returns[cut:], sigmas[cut:], weights[cut:]]

Minimum Risk Portfolio

In [8]:
def get_min_risk_portfolio(pdata_pct_m):
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    ef_dots = get_ef_dots(mu, cov_m)
    #Min Risk portfolio
    min_risk = [ef_dots[0][0], ef_dots[1][0], ef_dots[2][0]]
    return ef_dots, min_risk

Max Sharpe Portfolio

In [9]:
def get_max_sharpe_portfolio(pdata_pct_m):
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    ef_dots = get_ef_dots(mu, cov_m)
    #Max Sharpe portfolio
    sharpe_values = []
    for w in ef_dots[2]:
        sharpe_values.append( (w @ (mu.values - r_f)) / np.sqrt(w @ cov_m @ w) )
    max_sharpe = sharpe_values.index(max(sharpe_values))
    return ef_dots, [ef_dots[0][max_sharpe], ef_dots[1][max_sharpe], ef_dots[2][max_sharpe]]

Max Treynor Ratio Portfolio

In [10]:
from scipy.stats import beta, dweibull, gamma, gumbel_r, lognorm, norm

In [11]:
def median_cov(pdata_pct_m):
    median_cov_m = []
    for col in pdata_pct_m.columns:
        median_cov_m.append(np.median( np.abs(pdata_pct_m[col] - np.median(pdata_pct_m[col])) ))
    return np.diag(median_cov_m)

def spearman_corr(pdata_pct_m):
    return pdata_pct_m.corr(method='spearman')

def median_cov_spearman_corr_matrix(pdata_pct_m):
    median_cov_matrix = median_cov(pdata_pct_m)
    spearman_corr_matrix = spearman_corr(pdata_pct_m)
    
    cov_max_tr = median_cov_matrix @ spearman_corr_matrix @ median_cov_matrix
    cov_max_tr.columns = pdata_pct_m.columns
    cov_max_tr.index = pdata_pct_m.columns
    return cov_max_tr

#Maximum Likelihood estimation for first moment

def get_mom_from_mle(x, n=1):
 
    def beta_fit(x):
        a,b,c,d = beta.fit(x, method="MLE")
        mle = np.log(beta.pdf(x, a, b,c ,d)).sum()
        mom1 = beta.stats(a, b, c, d, moments='mvsk')[0]
        mom2 = beta.stats(a, b, c, d, moments='mvsk')[1]
        return [mle, mom1, mom2]

    def dweibull_fit(x):
        a,b,c = dweibull.fit(x, method="MLE")
        mle = np.log(dweibull.pdf(x, a, b,c )).sum()
        mom1 = dweibull.stats(a, b, c, moments='mvsk')[0]
        mom2 = dweibull.stats(a, b, c, moments='mvsk')[1]
        return [mle, mom1, mom2]

    def gamma_fit(x):
        a,b,c = gamma.fit(x, method="MLE")
        mle = np.log(gamma.pdf(x, a, b,c )).sum()
        mom1 = gamma.stats(a, b, c, moments='mvsk')[0]
        mom2 = gamma.stats(a, b, c, moments='mvsk')[1]
        return [mle, mom1, mom2]

    def gumbel_fit(x):
        a,b = gumbel_r.fit(x, method="MLE")
        mle = np.log(gumbel_r.pdf(x, a, b)).sum()
        mom1 = gumbel_r.stats(a, b, moments='mvsk')[0]
        mom2 = gumbel_r.stats(a, b, moments='mvsk')[1]
        return [mle, mom1, mom2]

    def lognorm_fit(x):
        a,b,c = lognorm.fit(x, method="MLE")
        mle = np.log(lognorm.pdf(x, a, b,c)).sum()
        mom1 = lognorm.stats(a, b, c, moments='mvsk')[0]
        mom2 = lognorm.stats(a, b, c, moments='mvsk')[1]
        return [mle, mom1, mom2]

    def norm_fit(x):
        a,b = norm.fit(x, method="MLE")
        mle = np.log(norm.pdf(x, a, b)).sum()
        mom1 = norm.stats(a, b, moments='mvsk')[0]
        mom2 = norm.stats(a, b, moments='mvsk')[1]
        return [mle, mom1, mom2]
    
    try: 
        bf = beta_fit(x)
    except:
        bf = [0, np.mean(x), np.var(x)]
    
    result = [bf, dweibull_fit(x), gamma_fit(x), gumbel_fit(x), lognorm_fit(x), norm_fit(x)]
    result = max(result, key=lambda x: x[0])
    if n == 1:
        return result[1]
    if n == 2:
        return result[2]
    
def exp_ret_from_mle(pdata_pct_m):
    exp_ret_from_mle_vector = []
    for col in pdata_pct_m.columns:
        data = pdata_pct_m[col].values
        exp_ret_from_mle_vector.append( get_mom_from_mle(data, n=1) )
    for j, i in enumerate(exp_ret_from_mle_vector):
        if i > 1:
            exp_ret_from_mle_vector[j] = 0.005
    exp_ret_from_mle_vector = pd.Series(exp_ret_from_mle_vector, index=pdata_pct_m.columns)
    return exp_ret_from_mle_vector

def get_asset_betas(pdata_pct_m, msci_y):
    betas = []
    for col in pdata_pct_m.columns:
        betas.append( np.cov(msci_y.values, pdata_pct_m[col], ddof=0)[0][1] / np.var(msci_y) )
    return np.array(betas)

def get_max_treynor_portfolio(pdata_pct_m, msci_y):
    mu = exp_ret_from_mle(pdata_pct_m)
    cov_m = median_cov_spearman_corr_matrix(pdata_pct_m)
    ef_dots = get_ef_dots(mu, cov_m)
    #Max Treynor portfolio
    treynor_values = []
    betas = get_asset_betas(pdata_pct_m, msci_y)
    for w in ef_dots[2]:
        treynor_values.append( (mu - r_f) @ w / (w @ betas.T) )
    max_treynor = treynor_values.index(max(treynor_values))
    return ef_dots, [ef_dots[0][max_treynor], ef_dots[1][max_treynor], ef_dots[2][max_treynor]], mu, cov_m

Maximum Sortino Portfolio

In [12]:
def capm_exp_ret(pdata_pct_m, msci_y, r_f):
    mu = []
    for i, col in enumerate(pdata_pct_m.columns):
        beta_reg, alpha_reg = np.polyfit(x = pdata_pct_m[col].values[1:] , y = msci_y.pct_change().values[1:] , deg = 1)
        mu.append(alpha_reg - r_f + beta_reg * (msci_y.pct_change().mean() - r_f))
    if len([i for i in mu if i > 0]) < 3:
        rnd = np.random.randint(0, 200, 3)
        mu[rnd[0]] = np.random.uniform(0.001, 0.02) 
        mu[rnd[1]] = np.random.uniform(0.001, 0.02) 
        mu[rnd[2]] = np.random.uniform(0.001, 0.02) 
    return pd.Series(mu, index=pdata_pct_m.columns)

def semi_cov_matrix(pdata_pct_m, r_f):
    semi_cov = []
    for col in pdata_pct_m.columns:
        semi_cov.append((np.minimum(pdata_pct_m[col] - r_f, 0)**2).mean())
    return np.diag(semi_cov)

def kendall_corr(pdata_pct_m):
    return pdata_pct_m.corr(method='kendall')

def semi_cov_kendall_corr_matrix(pdata_pct_m, r_f):
    cov_m = semi_cov_matrix(pdata_pct_m, r_f) @ kendall_corr(pdata_pct_m) @ semi_cov_matrix(pdata_pct_m, r_f)
    cov_m.index = pdata_pct_m.columns
    cov_m.columns = pdata_pct_m.columns
    return cov_m

def get_max_sortino_portfolio(pdata_pct_m, msci_y, r_f):
    mu = capm_exp_ret(pdata_pct_m, msci_y, r_f)
    cov_m = semi_cov_kendall_corr_matrix(pdata_pct_m, r_f)
    ef_dots = get_ef_dots(mu, cov_m)
    #Max Sortino portfolio
    sortino_values = []
    for w in ef_dots[2]:
        sortino_values.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) )
    max_sortino = sortino_values.index(max(sortino_values))
    return ef_dots, [ef_dots[0][max_sortino], ef_dots[1][max_sortino], ef_dots[2][max_sortino]], mu, cov_m

Maximum Stuzer Ratio Portfolio

In [13]:
from statsmodels.tsa.arima.model import ARIMA
from IPython.display import clear_output
from tqdm import tqdm
from scipy.optimize import minimize

In [14]:
def arma_exp_ret(pdata, pdata_pct):
    mu = []
    for i in tqdm(pdata_pct.columns):
        model = ARIMA(pdata_pct[i], order=(9, 0, 7), freq='D')
        model_fit = model.fit()
        start = len(pdata_pct[i])
        end = start + 30
        predictions = model_fit.predict(start=start, end=end)
        clear_output(wait=True)
        prev_v = pdata[i][-1]
        for j in predictions:
            if j != 0:
                prev_v = (j + 1) * prev_v
            else:
                pass
        mu.append((prev_v - pdata[i][-1]) / pdata[i][-1])
    return pd.Series(mu, index=pdata_pct.columns)

def var_from_mle(pdata_pct_m):
    var = []
    for col in pdata_pct_m.columns:
        x = pdata_pct_m[col].values
        var.append( get_mom_from_mle(x, n=2) )
    return np.diag(var)

def arch_copula_corr(p=0):
    archimedian_cor= np.loadtxt('correlation_analysis/intermediate_data/archimedian/archimedian_cors%s.gz' %p, delimiter=',')
    return archimedian_cor

def var_mle_arch_copula_corr_matrix(pdata_pct_m, p=0):
    var = var_from_mle(pdata_pct_m)
    corr_m = arch_copula_corr(p=p)
    cov_m = pd.DataFrame(var @ corr_m @ var)
    cov_m.index = pdata_pct_m.columns
    cov_m.columns = pdata_pct_m.columns
    return cov_m.round(6)

def get_max_stuzer_portfolio(pdata, pdata_pct, pdata_pct_m, r_f, p=0):
    mu = arma_exp_ret(pdata, pdata_pct)
    cov_m = var_mle_arch_copula_corr_matrix(pdata_pct_m, p=p)
    ef_dots = get_ef_dots(mu, cov_m)
    #Max Stuzer Portfolio
    def st_f(I, mu, r_f, w):
        return -np.log(np.mean(np.exp(I * w @ (mu - r_f))))
    stuzer_values = []
    for w in ef_dots[2]:
        res = minimize(lambda I: -st_f(I, mu, r_f, w), x0=1.0, method='BFGS')
        I_st = res.x[0]
        stuzer_values.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) * np.sqrt(2 * abs(I_st)) )
    max_stuzer = stuzer_values.index(max(stuzer_values))
    clear_output(wait=True)
    return ef_dots, [ef_dots[0][max_stuzer], ef_dots[1][max_stuzer], ef_dots[2][max_stuzer]], mu, cov_m

Min VaR Portfolio (delta-norm method)

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.neighbors import KernelDensity

In [16]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, preq_len):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.preq_len = preq_len
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h0=None, c0=None):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = out[:, -30:, :]
        out = self.fc(out)
        return out
    
def eval_lstm_model():
    input_size = 200
    hidden_size = 256
    num_layers = 2
    output_size = 200
    preq_len = 30
    
    model = LSTMModel(input_size, hidden_size, num_layers, input_size, preq_len).to(device)
    model.load_state_dict(torch.load('models/lstm_predict.pth', weights_only=True))
    model.eval()
    return model

def lstm_exp_ret(pdata_pct):
    model = eval_lstm_model()
    clear_output(wait=True)
    pre = np.array([pdata_pct.values.astype(np.float32)])
    predicted = model(torch.from_numpy(pre))
    predicted = predicted.detach().numpy()
    predicted = predicted.reshape(predicted.shape[1], predicted.shape[2])
    predicted = pd.DataFrame(predicted)
    predicted = predicted.mean()
    predicted.index = pdata_pct.columns
    return predicted

def var_from_kde(pdata_pct):
    var = []
    for col in pdata_pct.columns:
        kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(pd.DataFrame(pdata_pct[col]))
        samples = kde.sample(n_samples=1000)
        var_col = pd.Series([i[0] for i in samples]).var()
        var.append(var_col)
    return np.diag(var)

def glasso_corr(p=0):
    glasso_cor = np.loadtxt('correlation_analysis/intermediate_data/glasso_cors/glasso_cors%s.gz' %p, delimiter=',')
    return glasso_cor

def var_kde_glasso_corr_matrix(pdata_pct_m, p=0):
    var = var_from_kde(pdata_pct_m)
    corr_m = glasso_corr(p=p)
    cov_m = pd.DataFrame(var @ corr_m @ var)
    cov_m.index = pdata_pct_m.columns
    cov_m.columns = pdata_pct_m.columns
    return cov_m

def get_min_var_portfolio(pdata, pdata_pct, pdata_pct_m, p=0, alpha=0.95, days=30):
    mu = lstm_exp_ret(pdata_pct)
    cov_m = var_kde_glasso_corr_matrix(pdata_pct_m, p=p)
    ef_dots = get_ef_dots(mu, cov_m)
    value_at_risk = []
    for w in ef_dots[2]:
        portfolio_ret = w @ mu
        portfolio_vol = np.sqrt(w @ cov_m @ w )
        var = abs((portfolio_ret * days - abs(norm.ppf(1-alpha)) * portfolio_vol * np.sqrt(days)))
        value_at_risk.append( var )
    min_var = value_at_risk.index(min(value_at_risk))
    return ef_dots, [ef_dots[0][min_var], ef_dots[1][min_var], ef_dots[2][min_var]], mu, cov_m, value_at_risk

Minimum CVaR Portfolio (historical modelling)

In [17]:
from arch import arch_model
from sklearn.preprocessing import MinMaxScaler

In [18]:
class CNNTimeSeriesForecaster(nn.Module):
    def __init__(self, input_size=365, output_size=30):
        super(CNNTimeSeriesForecaster, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Рассчитываем размер после пулинга
        self.flatten_size = 128 * (input_size // 8)
        
        self.fc1 = nn.Linear(self.flatten_size, 256)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(256, output_size)
    
    def forward(self, x):
        x = x.unsqueeze(1)
        
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        
        return x
    
def eval_cnn_model(index):
    model = CNNTimeSeriesForecaster(input_size=365, output_size=30).to(device)
    model.load_state_dict(torch.load('models/cnn_models/cnn%s.pth' %index, weights_only=True))
    model.eval()
    return model

def cnn_asset_exp_ret(index, passet_returns):
    model = eval_cnn_model(index)
    clear_output(wait=True)
    pre = np.array([passet_returns.values.astype(np.float32)])
    predicted = model(torch.from_numpy(pre))
    predicted = predicted.detach().numpy()[0]
    return predicted[-1]

def cnn_exp_ret(pdata_pct):
    mu = []
    for index, i in enumerate(pdata_pct.columns):
        mu.append(cnn_asset_exp_ret(index, pdata_pct[i]))
    return pd.Series(mu, index=pdata_pct.columns)

def var_from_garch(pdata_pct):
    var = []
    for col in pdata_pct.columns:
        model = arch_model(pdata_pct[col], vol='Garch', p=1, q=1)
        results = model.fit(update_freq=5)
        forecast = results.forecast(horizon=30)
        var.append(np.sqrt(forecast.variance.values.sum()))
        clear_output(wait=True)
    return var

def kpca_corr(p=0):
    kpca_cor = np.loadtxt('correlation_analysis/intermediate_data/kpca/X_kcpa_copula_cor_large_list2%s.gz' %p, delimiter=',')
    scaller = MinMaxScaler(feature_range=(-1, 1))
    kpca_cor = scaller.fit_transform(kpca_cor)
    return np.corrcoef(kpca_cor)

def var_garch_kpca_corr_matrix(pdata_pct_m, p=0):
    var = var_from_garch(pdata_pct)
    corr_m = kpca_corr(p=p)
    cov_m = pd.DataFrame(np.diag(var) @ corr_m @ np.diag(var))
    cov_m.index = pdata_pct_m.columns
    cov_m.columns = pdata_pct_m.columns
    return cov_m

def get_min_cvar_portfolio(pdata_pct, p=1, alpha=0.95):
    mu = cnn_exp_ret(pdata_pct)
    cov_m = var_garch_kpca_corr_matrix(pdata_pct_m, p=p)
    ef_dots = get_ef_dots(mu, cov_m)
    var_p = []
    cvar_p = []
    for w in ef_dots[2]:
        portfolio_ret = pdata_pct @ w
        var_level = int((1 - alpha) * len(mu))
        sorted_ret = np.sort(portfolio_ret)
        var = -sorted_ret[var_level]
        cvar = -np.mean(sorted_ret[:var_level])
        var_p.append(var)
        cvar_p.append(cvar)
    min_cvar = cvar_p.index(min(cvar_p))
    return ef_dots, [ef_dots[0][min_cvar], ef_dots[1][min_cvar], ef_dots[2][min_cvar]], mu, cov_m, cvar_p

Maximum r/CVaR Portfolio (Monte Carlo modelling)

In [19]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

In [20]:
def xgboost_exp_ret(pdata):
    mu = []
    for i in pdata.columns:
        X_train = np.array([pdata[i].values[:-30]])
        y_train = np.array([pdata[i].values[-30:]])
        X_test = np.array([pdata[i].values[30:]])
        
        model = XGBRegressor(n_estimators=100, learning_rate=0.1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mu.append((y_pred[0][-1] - X_test[0][-1]) / X_test[0][-1])
    return pd.Series(mu, index=pdata.columns) * 100

def var_from_lgbm(pdata):
    var = []
    preds = []
    
    def create_ds(data, ws, h):
        X, y = [], []
        for i in range(len(data) - ws - h + 1):
            X.append(data[i:i + ws])
            y.append(data[i + ws:i + ws + h])
        return np.array(X), np.array(y)
    
    for i in pdata.columns:
        X, y = create_ds(pdata[i].values, ws=60, h=30)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        model = LGBMRegressor(objective='regression'
                             , metric='rmse'
                             , boosting_type='gbdt'
                             , num_leaves=31
                             , learning_rate=0.01
                             , n_estimators=1000
                             , verbose=-1)

        model.fit(X_train, y_train[:, 0], eval_set=[(X_train, y_train[:, 0]),
                                                   (X_test, y_test[:, 0])])
        pred = model.predict(X_test)
        var.append(np.var(pd.Series(pred).pct_change()))
    clear_output(wait=True)   
    return np.array(var) * 1000

def encoder_corr(p=1):
    if p <= 41:
        x = np.loadtxt('correlation_analysis/intermediate_data/decoder/X_encoded_train_copula_large_list%s.gz' %p, delimiter=',')
    else:
        p = p - 42
        x = np.loadtxt('correlation_analysis/intermediate_data/decoder/X_encoded_val_copula_large_list%s.gz' %p, delimiter=',')
    return np.corrcoef(x)

def var_lgbm_encoder_corr(pdata, p=1):
    var = var_from_lgbm(pdata)
    corr_m = encoder_corr(p=p)
    cov_m = pd.DataFrame(np.diag(var) @ corr_m @ np.diag(var))
    cov_m.index = pdata.columns
    cov_m.columns = pdata.columns
    return cov_m

def get_r_cvar_portfolio(pdata, predicted_days=30, n_sim=1000, mu=None, sigma=None, corr_matrix=None, weight=[]):
    n_assets = pdata.shape[1]
    history_days = pdata.shape[0]
    predicted_days = predicted_days
    total_days = history_days + predicted_days
    n_sim = n_sim
    
    if mu is None:
        mu = pdata.mean().values
    if sigma is None:    
        sigma = np.diag(pdata.cov())
    if corr_matrix is None:
        corr_matrix = pdata.corr()
        
    theta = sigma / (2 * mu)
    
    S0 = pdata.values
    try:
        L = np.linalg.cholesky(corr_matrix)
    except:
        corr_matrix_new = corr_matrix + 1e-10 * np.eye(corr_matrix.shape[0])
        L = np.linalg.cholesky(corr_matrix_new)
        
    dt = 1/history_days
    sqrt_dt = np.sqrt(dt)
    
    paths = np.zeros((n_sim, total_days, n_assets))
    paths[:, :history_days, :] = S0
    
    for i in range(n_sim):
        Z = np.random.normal(0, 1, size=(predicted_days, n_assets))
        corr_Z = Z @ L.T

        for t in range(history_days, total_days):
            for k in range(n_assets):
                drift = theta[k] * (mu[k] - paths[i, t, k]) * dt
                diffusion = sigma[k] * sqrt_dt * corr_Z[t-history_days, k]
                paths[i, t, k] = paths[i, t-1, k] + drift + diffusion
                
    net_wealths = []
    for p in range(len(paths)):
        period_wealths = [sum((np.array(weight) * 1000000) / paths[p][-31:][0])]
        p0 = [paths[p][-31:][0]]
        qnt0 = ((np.array(weight) * 1000000) / paths[p][-31:][0] ).astype(int)
        for day in range(31):
            p1 = paths[p][-31:][day]
            w1 = sum(qnt0 * p1)
            period_wealths.append(w1)
        net_wealths.append(period_wealths)
        
    cvars = []
    for i in range(len(net_wealths)):
        cvar = np.mean(np.sort((np.array(net_wealths[i][1:]) / 1000000) - 1)[:3])
        cvars.append(cvar)
        
    pseudo_portfolio_returns = []
    for i in range(len(net_wealths)):
        pseudo_portfolio_returns.append(net_wealths[i][-1] / 1000000 * 100 - 100)
        
    return np.mean(pseudo_portfolio_returns), np.mean(cvars)

def get_max_r_cvar_portfolio(pdata, p=1):
    mu = xgboost_exp_ret(pdata)
    cov_m = var_lgbm_encoder_corr(pdata, p=p)
    ef_dots = get_ef_dots(mu, cov_m)
    pseudo_returns = []
    pseudo_cvars = []
    pseudo_r_cvar = []
    pseudo_r_cvar_all = []
    for w in ef_dots[2]:
        pseudo_ret, pseudo_cvar = get_r_cvar_portfolio(pdata
                                                       , predicted_days=30
                                                       , n_sim=1000
                                                       , mu=None
                                                       , sigma=None
                                                       , corr_matrix=None
                                                       , weight=w)
        pseudo_returns.append(pseudo_ret)
        pseudo_cvars.append(pseudo_cvar)
        pseudo_r_cvar.append(pseudo_ret / -pseudo_cvar)
        pseudo_r_cvar_all.append([pseudo_ret, pseudo_cvar])
    max_r_cvar = pseudo_r_cvar.index(max(pseudo_r_cvar))
    return ef_dots, [ef_dots[0][max_r_cvar], ef_dots[1][max_r_cvar], ef_dots[2][max_r_cvar]], mu, cov_m, pseudo_r_cvar, pseudo_r_cvar_all

Implementation

In [21]:
all_results = []

In [22]:
%%time

# 1 portfolio

portfolio_return_itg = [] #
portfolio_return_per = [] #
portfolio_wealth_per = [] #
portfolio_cov_per = [] #

portfolio_sharpe_per = [] #
portfolio_treynor_per = [] #
portfolio_sortino_per = [] #
portfolio_stuzer_per = [] #
portfolio_modiliani_per = [] #
portfolio_var_per = [] #
portfolio_cvar_per = [] #
portfolio_r_cvar_per = [] #
portfolio_alpha_per = [] #

portfolio_weights = [] #

for i in range(61):
    pdata = period_data[i]
    pdata_pct = period_data_pct[i]
    pdata_pct_m = period_data_pct_m[i]
    r_f = period_rate[i]
    msci_y = period_msci[i]
    
    print(i)
    print('start')
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    _, portfolio = get_min_risk_portfolio(pdata_pct_m)
    
    #return, cov, weights
    print(1)
    portfolio_return_per.append(portfolio[0])
    portfolio_cov_per.append(portfolio[1])
    portfolio_weights.append(portfolio[2])
    w = np.array(portfolio[2])
    
    #wealth periodic
    print(2)
    if i == 0:
        quant = (w * 1000000 /  pdata.iloc[-1].values).astype(int)
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
    else:
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
        quant = (w * portfolio_wealth_per[-1] /  pdata.iloc[-1].values).astype(int) 
    
    #itg return
    print(3)
    if i == 60:
        portfolio_return_itg.append(portfolio_wealth_per[-1] / portfolio_wealth_per[0])
        
    #sharpe
    print(4)
    portfolio_sharpe_per.append( (w @ (mu.values - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #treynor
    print(5)
    betas = get_asset_betas(pdata_pct_m, msci_y['MSCI'])
    portfolio_treynor_per.append( (mu - r_f) @ w / (w @ betas.T) )
    
    #sortino
    print(6)
    portfolio_sortino_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #stuzer
    print(7)
    def st_f(I, mu, r_f, w):
        return -np.log(np.mean(np.exp(I * w @ (mu - r_f))))
    res = minimize(lambda I: -st_f(I, mu, r_f, w), x0=1.0, method='BFGS')
    I_st = res.x[0]
    portfolio_stuzer_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) * np.sqrt(2 * abs(I_st)) )
    
    #modiliani
    print(8)
    portfolio_modiliani_per.append((portfolio[0] - r_f) * (np.var(msci_y['MSCI']) / np.var(portfolio_return_per)) + r_f)
    
    #var
    print(9)
    alpha=0.95
    days=30
    var = abs(portfolio[0] * days - abs(norm.ppf(1-alpha)) * portfolio[1] * np.sqrt(days)) / 100
    portfolio_var_per.append(portfolio_wealth_per[-1] * var)
    
    #cvar
    print(10)
    alpha=0.95
    portfolio_ret = pdata_pct @ w
    var_level = int((1 - alpha) * len(mu))
    sorted_ret = np.sort(portfolio_ret)
    var = -sorted_ret[var_level]
    cvar = -np.mean(sorted_ret[:var_level])
    portfolio_cvar_per.append(portfolio_wealth_per[-1] * cvar)
    
    #r/cvar
    print(11)
    pseudo_ret, pseudo_cvar = get_r_cvar_portfolio(pdata
                                                   , predicted_days=30
                                                   , n_sim=1000
                                                   , mu=None
                                                   , sigma=None
                                                   , corr_matrix=None
                                                   , weight=w)
    portfolio_r_cvar_per.append(pseudo_ret / -pseudo_cvar)
    
    #alpha
    print(12)
    r_m = (msci_y.iloc[0, 0] - msci_y.iloc[-1, 0]) / msci_y.iloc[-1, 0]
    if i == 0:
        b_p = 0
    else:
        msci_y_pct = msci_y.pct_change().dropna()['MSCI']
        if len(portfolio_return_per) <= len(msci_y_pct):
            x = len(portfolio_return_per)
            b_p = np.cov(portfolio_return_per, msci_y_pct[:x])[0][0] / np.var(msci_y_pct)
        else:
            x = len(msci_y_pct)
            b_p = np.cov(portfolio_return_per[-x:], msci_y_pct)[0][0] / np.var(msci_y_pct)
        
    portfolio_alpha_per.append(portfolio[0] - (r_f + (r_m - r_f) * b_p))
    
    clear_output(wait=True)
    
all_results.append([portfolio_return_itg
                    , portfolio_return_per
                    , portfolio_wealth_per
                    , portfolio_cov_per
                    , portfolio_sharpe_per # / 100
                    , portfolio_treynor_per # / 1000000
                    , portfolio_sortino_per # / 100
                    , portfolio_stuzer_per # / 10000
                    , portfolio_modiliani_per # / 10000000
                    , portfolio_var_per
                    , portfolio_cvar_per 
                    , portfolio_r_cvar_per # / 100
                    , portfolio_alpha_per
                    , portfolio_weights])

CPU times: total: 10min 37s
Wall time: 9min 40s


In [23]:
%%time

# 2 portfolio

portfolio_return_itg = [] #
portfolio_return_per = [] #
portfolio_wealth_per = [] #
portfolio_cov_per = [] #

portfolio_sharpe_per = [] #
portfolio_treynor_per = [] #
portfolio_sortino_per = [] #
portfolio_stuzer_per = [] #
portfolio_modiliani_per = [] #
portfolio_var_per = [] #
portfolio_cvar_per = [] #
portfolio_r_cvar_per = [] #
portfolio_alpha_per = [] #

portfolio_weights = [] #

for i in range(61):
    pdata = period_data[i]
    pdata_pct = period_data_pct[i]
    pdata_pct_m = period_data_pct_m[i]
    r_f = period_rate[i]
    msci_y = period_msci[i]
    
    print(i)
    print('start')
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    _, portfolio = get_max_sharpe_portfolio(pdata_pct_m)
    
    #return, cov, weights
    print(1)
    portfolio_return_per.append(portfolio[0])
    portfolio_cov_per.append(portfolio[1])
    portfolio_weights.append(portfolio[2])
    w = np.array(portfolio[2])
    
    #wealth periodic
    print(2)
    if i == 0:
        quant = (w * 1000000 /  pdata.iloc[-1].values).astype(int)
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
    else:
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
        quant = (w * portfolio_wealth_per[-1] /  pdata.iloc[-1].values).astype(int) 
    
    #itg return
    print(3)
    if i == 60:
        portfolio_return_itg.append(portfolio_wealth_per[-1] / portfolio_wealth_per[0])
        
    #sharpe
    print(4)
    portfolio_sharpe_per.append( (w @ (mu.values - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #treynor
    print(5)
    betas = get_asset_betas(pdata_pct_m, msci_y['MSCI'])
    portfolio_treynor_per.append( (mu - r_f) @ w / (w @ betas.T) )
    
    #sortino
    print(6)
    portfolio_sortino_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #stuzer
    print(7)
    def st_f(I, mu, r_f, w):
        return -np.log(np.mean(np.exp(I * w @ (mu - r_f))))
    res = minimize(lambda I: -st_f(I, mu, r_f, w), x0=1.0, method='BFGS')
    I_st = res.x[0]
    portfolio_stuzer_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) * np.sqrt(2 * abs(I_st)) )
    
    #modiliani
    print(8)
    portfolio_modiliani_per.append((portfolio[0] - r_f) * (np.var(msci_y['MSCI']) / np.var(portfolio_return_per)) + r_f)
    
    #var
    print(9)
    alpha=0.95
    days=30
    var = abs(portfolio[0] * days - abs(norm.ppf(1-alpha)) * portfolio[1] * np.sqrt(days)) / 100
    portfolio_var_per.append(portfolio_wealth_per[-1] * var)
    
    #cvar
    print(10)
    alpha=0.95
    portfolio_ret = pdata_pct @ w
    var_level = int((1 - alpha) * len(mu))
    sorted_ret = np.sort(portfolio_ret)
    var = -sorted_ret[var_level]
    cvar = -np.mean(sorted_ret[:var_level])
    portfolio_cvar_per.append(portfolio_wealth_per[-1] * cvar)
    
    #r/cvar
    print(11)
    pseudo_ret, pseudo_cvar = get_r_cvar_portfolio(pdata
                                                   , predicted_days=30
                                                   , n_sim=1000
                                                   , mu=None
                                                   , sigma=None
                                                   , corr_matrix=None
                                                   , weight=w)
    portfolio_r_cvar_per.append(pseudo_ret / -pseudo_cvar)
    
    #alpha
    print(12)
    r_m = (msci_y.iloc[0, 0] - msci_y.iloc[-1, 0]) / msci_y.iloc[-1, 0]
    if i == 0:
        b_p = 0
    else:
        msci_y_pct = msci_y.pct_change().dropna()['MSCI']
        if len(portfolio_return_per) <= len(msci_y_pct):
            x = len(portfolio_return_per)
            b_p = np.cov(portfolio_return_per, msci_y_pct[:x])[0][0] / np.var(msci_y_pct)
        else:
            x = len(msci_y_pct)
            b_p = np.cov(portfolio_return_per[-x:], msci_y_pct)[0][0] / np.var(msci_y_pct)
        
    portfolio_alpha_per.append(portfolio[0] - (r_f + (r_m - r_f) * b_p))
    
    clear_output(wait=True)
    
all_results.append([portfolio_return_itg
                    , portfolio_return_per
                    , portfolio_wealth_per
                    , portfolio_cov_per
                    , portfolio_sharpe_per # / 1000
                    , portfolio_treynor_per # / 1000000
                    , portfolio_sortino_per # / 100
                    , portfolio_stuzer_per # / 100000
                    , portfolio_modiliani_per # / 10000000
                    , portfolio_var_per
                    , portfolio_cvar_per 
                    , portfolio_r_cvar_per # / 100
                    , portfolio_alpha_per
                    , portfolio_weights])

CPU times: total: 10min 41s
Wall time: 9min 42s


In [24]:
%%time

# 3 portfolio

portfolio_return_itg = [] #
portfolio_return_per = [] #
portfolio_wealth_per = [] #
portfolio_cov_per = [] #

portfolio_sharpe_per = [] #
portfolio_treynor_per = [] #
portfolio_sortino_per = [] #
portfolio_stuzer_per = [] #
portfolio_modiliani_per = [] #
portfolio_var_per = [] #
portfolio_cvar_per = [] #
portfolio_r_cvar_per = [] #
portfolio_alpha_per = [] #

portfolio_weights = [] #

for i in range(61):
    pdata = period_data[i]
    pdata_pct = period_data_pct[i]
    pdata_pct_m = period_data_pct_m[i]
    r_f = period_rate[i]
    msci_y = period_msci[i]
    
    print(i)
    print('start')
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    _, portfolio, _, _ = get_max_treynor_portfolio(pdata_pct_m, msci_y['MSCI'])
    
    #return, cov, weights
    print(1)
    portfolio_return_per.append(portfolio[0])
    portfolio_cov_per.append(portfolio[1])
    portfolio_weights.append(portfolio[2])
    w = np.array(portfolio[2])
    
    #wealth periodic
    print(2)
    if i == 0:
        quant = (w * 1000000 /  pdata.iloc[-1].values).astype(int)
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
    else:
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
        quant = (w * portfolio_wealth_per[-1] /  pdata.iloc[-1].values).astype(int) 
    
    #itg return
    print(3)
    if i == 60:
        portfolio_return_itg.append(portfolio_wealth_per[-1] / portfolio_wealth_per[0])
        
    #sharpe
    print(4)
    portfolio_sharpe_per.append( (w @ (mu.values - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #treynor
    print(5)
    betas = get_asset_betas(pdata_pct_m, msci_y['MSCI'])
    portfolio_treynor_per.append( (mu - r_f) @ w / (w @ betas.T) )
    
    #sortino
    print(6)
    portfolio_sortino_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #stuzer
    print(7)
    def st_f(I, mu, r_f, w):
        return -np.log(np.mean(np.exp(I * w @ (mu - r_f))))
    res = minimize(lambda I: -st_f(I, mu, r_f, w), x0=1.0, method='BFGS')
    I_st = res.x[0]
    portfolio_stuzer_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) * np.sqrt(2 * abs(I_st)) )
    
    #modiliani
    print(8)
    portfolio_modiliani_per.append((portfolio[0] - r_f) * (np.var(msci_y['MSCI']) / np.var(portfolio_return_per)) + r_f)
    
    #var
    print(9)
    alpha=0.95
    days=30
    var = abs(portfolio[0] * days - abs(norm.ppf(1-alpha)) * portfolio[1] * np.sqrt(days)) / 100
    portfolio_var_per.append(portfolio_wealth_per[-1] * var)
    
    #cvar
    print(10)
    alpha=0.95
    portfolio_ret = pdata_pct @ w
    var_level = int((1 - alpha) * len(mu))
    sorted_ret = np.sort(portfolio_ret)
    var = -sorted_ret[var_level]
    cvar = -np.mean(sorted_ret[:var_level])
    portfolio_cvar_per.append(portfolio_wealth_per[-1] * cvar)
    
    #r/cvar
    print(11)
    pseudo_ret, pseudo_cvar = get_r_cvar_portfolio(pdata
                                                   , predicted_days=30
                                                   , n_sim=1000
                                                   , mu=None
                                                   , sigma=None
                                                   , corr_matrix=None
                                                   , weight=w)
    portfolio_r_cvar_per.append(pseudo_ret / -pseudo_cvar)
    
    #alpha
    print(12)
    r_m = (msci_y.iloc[0, 0] - msci_y.iloc[-1, 0]) / msci_y.iloc[-1, 0]
    if i == 0:
        b_p = 0
    else:
        msci_y_pct = msci_y.pct_change().dropna()['MSCI']
        if len(portfolio_return_per) <= len(msci_y_pct):
            x = len(portfolio_return_per)
            b_p = np.cov(portfolio_return_per, msci_y_pct[:x])[0][0] / np.var(msci_y_pct)
        else:
            x = len(msci_y_pct)
            b_p = np.cov(portfolio_return_per[-x:], msci_y_pct)[0][0] / np.var(msci_y_pct)
        
    portfolio_alpha_per.append(portfolio[0] - (r_f + (r_m - r_f) * b_p))
    
    clear_output(wait=True)
    
all_results.append([portfolio_return_itg
                    , portfolio_return_per
                    , portfolio_wealth_per
                    , portfolio_cov_per
                    , portfolio_sharpe_per # / 10
                    , portfolio_treynor_per # / 10000000
                    , portfolio_sortino_per # / 10
                    , portfolio_stuzer_per # / 10000
                    , portfolio_modiliani_per # / 10000000
                    , portfolio_var_per # / 10
                    , portfolio_cvar_per 
                    , portfolio_r_cvar_per # / 100
                    , portfolio_alpha_per
                    , portfolio_weights])

CPU times: total: 40min 24s
Wall time: 39min 23s


In [25]:
%%time

# 4 portfolio

portfolio_return_itg = [] #
portfolio_return_per = [] #
portfolio_wealth_per = [] #
portfolio_cov_per = [] #

portfolio_sharpe_per = [] #
portfolio_treynor_per = [] #
portfolio_sortino_per = [] #
portfolio_stuzer_per = [] #
portfolio_modiliani_per = [] #
portfolio_var_per = [] #
portfolio_cvar_per = [] #
portfolio_r_cvar_per = [] #
portfolio_alpha_per = [] #

portfolio_weights = [] #

for i in range(61):
    pdata = period_data[i]
    pdata_pct = period_data_pct[i]
    pdata_pct_m = period_data_pct_m[i]
    r_f = period_rate[i]
    msci_y = period_msci[i]
    
    print(i)
    print('start')
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    _, portfolio, _, _ = get_max_sortino_portfolio(pdata_pct_m, msci_y['MSCI'], r_f)
    
    #return, cov, weights
    print(1)
    portfolio_return_per.append(portfolio[0])
    portfolio_cov_per.append(portfolio[1])
    portfolio_weights.append(portfolio[2])
    w = np.array(portfolio[2])
    
    #wealth periodic
    print(2)
    if i == 0:
        quant = (w * 1000000 /  pdata.iloc[-1].values).astype(int)
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
    else:
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
        quant = (w * portfolio_wealth_per[-1] /  pdata.iloc[-1].values).astype(int) 
    
    #itg return
    print(3)
    if i == 60:
        portfolio_return_itg.append(portfolio_wealth_per[-1] / portfolio_wealth_per[0])
        
    #sharpe
    print(4)
    portfolio_sharpe_per.append( (w @ (mu.values - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #treynor
    print(5)
    betas = get_asset_betas(pdata_pct_m, msci_y['MSCI'])
    portfolio_treynor_per.append( (mu - r_f) @ w / (w @ betas.T) )
    
    #sortino
    print(6)
    portfolio_sortino_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #stuzer
    print(7)
    def st_f(I, mu, r_f, w):
        return -np.log(np.mean(np.exp(I * w @ (mu - r_f))))
    res = minimize(lambda I: -st_f(I, mu, r_f, w), x0=1.0, method='BFGS')
    I_st = res.x[0]
    portfolio_stuzer_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) * np.sqrt(2 * abs(I_st)) )
    
    #modiliani
    print(8)
    portfolio_modiliani_per.append((portfolio[0] - r_f) * (np.var(msci_y['MSCI']) / np.var(portfolio_return_per)) + r_f)
    
    #var
    print(9)
    alpha=0.95
    days=30
    var = abs(portfolio[0] * days - abs(norm.ppf(1-alpha)) * portfolio[1] * np.sqrt(days)) / 100
    portfolio_var_per.append(portfolio_wealth_per[-1] * var)
    
    #cvar
    print(10)
    alpha=0.95
    portfolio_ret = pdata_pct @ w
    var_level = int((1 - alpha) * len(mu))
    sorted_ret = np.sort(portfolio_ret)
    var = -sorted_ret[var_level]
    cvar = -np.mean(sorted_ret[:var_level])
    portfolio_cvar_per.append(portfolio_wealth_per[-1] * cvar)
    
    #r/cvar
    print(11)
    pseudo_ret, pseudo_cvar = get_r_cvar_portfolio(pdata
                                                   , predicted_days=30
                                                   , n_sim=1000
                                                   , mu=None
                                                   , sigma=None
                                                   , corr_matrix=None
                                                   , weight=w)
    portfolio_r_cvar_per.append(pseudo_ret / -pseudo_cvar)
    
    #alpha
    print(12)
    r_m = (msci_y.iloc[0, 0] - msci_y.iloc[-1, 0]) / msci_y.iloc[-1, 0]
    if i == 0:
        b_p = 0
    else:
        msci_y_pct = msci_y.pct_change().dropna()['MSCI']
        if len(portfolio_return_per) <= len(msci_y_pct):
            x = len(portfolio_return_per)
            b_p = np.cov(portfolio_return_per, msci_y_pct[:x])[0][0] / np.var(msci_y_pct)
        else:
            x = len(msci_y_pct)
            b_p = np.cov(portfolio_return_per[-x:], msci_y_pct)[0][0] / np.var(msci_y_pct)
        
    portfolio_alpha_per.append(portfolio[0] - (r_f + (r_m - r_f) * b_p))
    
    clear_output(wait=True)
    
all_results.append([portfolio_return_itg
                    , portfolio_return_per
                    , portfolio_wealth_per
                    , portfolio_cov_per
                    , portfolio_sharpe_per # / 10
                    , portfolio_treynor_per # / 1000
                    , portfolio_sortino_per # / 10
                    , portfolio_stuzer_per # / 100
                    , portfolio_modiliani_per # / 10000000
                    , portfolio_var_per
                    , portfolio_cvar_per 
                    , portfolio_r_cvar_per # 
                    , portfolio_alpha_per
                    , portfolio_weights])

CPU times: total: 17min 30s
Wall time: 16min 30s


In [ ]:
%%time

# 5 portfolio

portfolio_return_itg = [] #
portfolio_return_per = [] #
portfolio_wealth_per = [] #
portfolio_cov_per = [] #

portfolio_sharpe_per = [] #
portfolio_treynor_per = [] #
portfolio_sortino_per = [] #
portfolio_stuzer_per = [] #
portfolio_modiliani_per = [] #
portfolio_var_per = [] #
portfolio_cvar_per = [] #
portfolio_r_cvar_per = [] #
portfolio_alpha_per = [] #

portfolio_weights = [] #

for i in range(0, 61):
    pdata = period_data[i]
    pdata_pct = period_data_pct[i]
    pdata_pct_m = period_data_pct_m[i]
    r_f = period_rate[i]
    msci_y = period_msci[i]
    
    print(i)
    print('start')
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    try:
        _, portfolio, _, _ = get_max_stuzer_portfolio(pdata, pdata_pct, pdata_pct_m, r_f, p=i)
    except:
        pass
    
    #return, cov, weights
    print(1)
    portfolio_return_per.append(portfolio[0])
    portfolio_cov_per.append(portfolio[1])
    portfolio_weights.append(portfolio[2])
    w = np.array(portfolio[2])
    
    #wealth periodic
    print(2)
    if i == 0:
        quant = (w * 1000000 /  pdata.iloc[-1].values).astype(int)
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
    else:
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
        quant = (w * portfolio_wealth_per[-1] /  pdata.iloc[-1].values).astype(int) 
    
    #itg return
    print(3)
    if i == 60:
        portfolio_return_itg.append(portfolio_wealth_per[-1] / portfolio_wealth_per[0])
        
    #sharpe
    print(4)
    portfolio_sharpe_per.append( (w @ (mu.values - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #treynor
    print(5)
    betas = get_asset_betas(pdata_pct_m, msci_y['MSCI'])
    portfolio_treynor_per.append( (mu - r_f) @ w / (w @ betas.T) )
    
    #sortino
    print(6)
    portfolio_sortino_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #stuzer
    print(7)
    def st_f(I, mu, r_f, w):
        return -np.log(np.mean(np.exp(I * w @ (mu - r_f))))
    res = minimize(lambda I: -st_f(I, mu, r_f, w), x0=1.0, method='BFGS')
    I_st = res.x[0]
    portfolio_stuzer_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) * np.sqrt(2 * abs(I_st)) )
    
    #modiliani
    print(8)
    portfolio_modiliani_per.append((portfolio[0] - r_f) * (np.var(msci_y['MSCI']) / np.var(portfolio_return_per)) + r_f)
    
    #var
    print(9)
    alpha=0.95
    days=30
    var = abs(portfolio[0] * days - abs(norm.ppf(1-alpha)) * portfolio[1] * np.sqrt(days)) / 100
    portfolio_var_per.append(portfolio_wealth_per[-1] * var)
    
    #cvar
    print(10)
    alpha=0.95
    portfolio_ret = pdata_pct @ w
    var_level = int((1 - alpha) * len(mu))
    sorted_ret = np.sort(portfolio_ret)
    var = -sorted_ret[var_level]
    cvar = -np.mean(sorted_ret[:var_level])
    portfolio_cvar_per.append(portfolio_wealth_per[-1] * cvar)
    
    #r/cvar
    print(11)
    pseudo_ret, pseudo_cvar = get_r_cvar_portfolio(pdata
                                                   , predicted_days=30
                                                   , n_sim=1000
                                                   , mu=None
                                                   , sigma=None
                                                   , corr_matrix=None
                                                   , weight=w)
    portfolio_r_cvar_per.append(pseudo_ret / -pseudo_cvar)
    
    #alpha
    print(12)
    r_m = (msci_y.iloc[0, 0] - msci_y.iloc[-1, 0]) / msci_y.iloc[-1, 0]
    if i == 0:
        b_p = 0
    else:
        msci_y_pct = msci_y.pct_change().dropna()['MSCI']
        if len(portfolio_return_per) <= len(msci_y_pct):
            x = len(portfolio_return_per)
            b_p = np.cov(portfolio_return_per, msci_y_pct[:x])[0][0] / np.var(msci_y_pct)
        else:
            x = len(msci_y_pct)
            b_p = np.cov(portfolio_return_per[-x:], msci_y_pct)[0][0] / np.var(msci_y_pct)
        
    portfolio_alpha_per.append(portfolio[0] - (r_f + (r_m - r_f) * b_p))
    
    clear_output(wait=True)
    
all_results.append([portfolio_return_itg
                    , portfolio_return_per
                    , portfolio_wealth_per
                    , portfolio_cov_per
                    , portfolio_sharpe_per # 
                    , portfolio_treynor_per # 
                    , portfolio_sortino_per # 
                    , portfolio_stuzer_per # 
                    , portfolio_modiliani_per # 
                    , portfolio_var_per
                    , portfolio_cvar_per 
                    , portfolio_r_cvar_per # 
                    , portfolio_alpha_per
                    , portfolio_weights])

In [26]:
%%time

# 6 portfolio

portfolio_return_itg = [] #
portfolio_return_per = [] #
portfolio_wealth_per = [] #
portfolio_cov_per = [] #

portfolio_sharpe_per = [] #
portfolio_treynor_per = [] #
portfolio_sortino_per = [] #
portfolio_stuzer_per = [] #
portfolio_modiliani_per = [] #
portfolio_var_per = [] #
portfolio_cvar_per = [] #
portfolio_r_cvar_per = [] #
portfolio_alpha_per = [] #

portfolio_weights = [] #

for i in range(61):
    pdata = period_data[i]
    pdata_pct = period_data_pct[i]
    pdata_pct_m = period_data_pct_m[i]
    r_f = period_rate[i]
    msci_y = period_msci[i]
    
    print(i)
    print('start')
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    _, portfolio, _, _, _ = get_min_var_portfolio(pdata, pdata_pct, pdata_pct_m, p=i, alpha=0.95, days=30)
    
    #return, cov, weights
    print(1)
    portfolio_return_per.append(portfolio[0])
    portfolio_cov_per.append(portfolio[1])
    portfolio_weights.append(portfolio[2])
    w = np.array(portfolio[2])
    
    #wealth periodic
    print(2)
    if i == 0:
        quant = (w * 1000000 /  pdata.iloc[-1].values).astype(int)
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
    else:
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
        quant = (w * portfolio_wealth_per[-1] /  pdata.iloc[-1].values).astype(int) 
    
    #itg return
    print(3)
    if i == 60:
        portfolio_return_itg.append(portfolio_wealth_per[-1] / portfolio_wealth_per[0])
        
    #sharpe
    print(4)
    portfolio_sharpe_per.append( (w @ (mu.values - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #treynor
    print(5)
    betas = get_asset_betas(pdata_pct_m, msci_y['MSCI'])
    portfolio_treynor_per.append( (mu - r_f) @ w / (w @ betas.T) )
    
    #sortino
    print(6)
    portfolio_sortino_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #stuzer
    print(7)
    def st_f(I, mu, r_f, w):
        return -np.log(np.mean(np.exp(I * w @ (mu - r_f))))
    res = minimize(lambda I: -st_f(I, mu, r_f, w), x0=1.0, method='BFGS')
    I_st = res.x[0]
    portfolio_stuzer_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) * np.sqrt(2 * abs(I_st)) )
    
    #modiliani
    print(8)
    portfolio_modiliani_per.append((portfolio[0] - r_f) * (np.var(msci_y['MSCI']) / np.var(portfolio_return_per)) + r_f)
    
    #var
    print(9)
    alpha=0.95
    days=30
    var = abs(portfolio[0] * days - abs(norm.ppf(1-alpha)) * portfolio[1] * np.sqrt(days)) / 100
    portfolio_var_per.append(portfolio_wealth_per[-1] * var)
    
    #cvar
    print(10)
    alpha=0.95
    portfolio_ret = pdata_pct @ w
    var_level = int((1 - alpha) * len(mu))
    sorted_ret = np.sort(portfolio_ret)
    var = -sorted_ret[var_level]
    cvar = -np.mean(sorted_ret[:var_level])
    portfolio_cvar_per.append(portfolio_wealth_per[-1] * cvar)
    
    #r/cvar
    print(11)
    pseudo_ret, pseudo_cvar = get_r_cvar_portfolio(pdata
                                                   , predicted_days=30
                                                   , n_sim=1000
                                                   , mu=None
                                                   , sigma=None
                                                   , corr_matrix=None
                                                   , weight=w)
    portfolio_r_cvar_per.append(pseudo_ret / -pseudo_cvar)
    
    #alpha
    print(12)
    r_m = (msci_y.iloc[0, 0] - msci_y.iloc[-1, 0]) / msci_y.iloc[-1, 0]
    if i == 0:
        b_p = 0
    else:
        msci_y_pct = msci_y.pct_change().dropna()['MSCI']
        if len(portfolio_return_per) <= len(msci_y_pct):
            x = len(portfolio_return_per)
            b_p = np.cov(portfolio_return_per, msci_y_pct[:x])[0][0] / np.var(msci_y_pct)
        else:
            x = len(msci_y_pct)
            b_p = np.cov(portfolio_return_per[-x:], msci_y_pct)[0][0] / np.var(msci_y_pct)
        
    portfolio_alpha_per.append(portfolio[0] - (r_f + (r_m - r_f) * b_p))
    
    clear_output(wait=True)
    
all_results.append([portfolio_return_itg
                    , portfolio_return_per
                    , portfolio_wealth_per
                    , portfolio_cov_per
                    , portfolio_sharpe_per # 
                    , portfolio_treynor_per # / 100000
                    , portfolio_sortino_per # 
                    , portfolio_stuzer_per # / 100
                    , portfolio_modiliani_per # / 10000000000
                    , portfolio_var_per # * 10
                    , portfolio_cvar_per 
                    , portfolio_r_cvar_per # 
                    , portfolio_alpha_per
                    , portfolio_weights])

CPU times: total: 13min 3s
Wall time: 10min 22s


In [27]:
%%time

# 7 portfolio

portfolio_return_itg = [] #
portfolio_return_per = [] #
portfolio_wealth_per = [] #
portfolio_cov_per = [] #

portfolio_sharpe_per = [] #
portfolio_treynor_per = [] #
portfolio_sortino_per = [] #
portfolio_stuzer_per = [] #
portfolio_modiliani_per = [] #
portfolio_var_per = [] #
portfolio_cvar_per = [] #
portfolio_r_cvar_per = [] #
portfolio_alpha_per = [] #

portfolio_weights = [] #

for i in range(61):
    pdata = period_data[i]
    pdata_pct = period_data_pct[i]
    pdata_pct_m = period_data_pct_m[i]
    r_f = period_rate[i]
    msci_y = period_msci[i]
    
    print(i)
    print('start')
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    _, portfolio, _, _, _ = get_min_cvar_portfolio(pdata_pct, p=i, alpha=0.95)
    
    #return, cov, weights
    print(1)
    portfolio_return_per.append(portfolio[0])
    portfolio_cov_per.append(portfolio[1])
    portfolio_weights.append(portfolio[2])
    w = np.array(portfolio[2])
    
    #wealth periodic
    print(2)
    if i == 0:
        quant = (w * 1000000 /  pdata.iloc[-1].values).astype(int)
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
    else:
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
        quant = (w * portfolio_wealth_per[-1] /  pdata.iloc[-1].values).astype(int) 
    
    #itg return
    print(3)
    if i == 60:
        portfolio_return_itg.append(portfolio_wealth_per[-1] / portfolio_wealth_per[0])
        
    #sharpe
    print(4)
    portfolio_sharpe_per.append( (w @ (mu.values - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #treynor
    print(5)
    betas = get_asset_betas(pdata_pct_m, msci_y['MSCI'])
    portfolio_treynor_per.append( (mu - r_f) @ w / (w @ betas.T) )
    
    #sortino
    print(6)
    portfolio_sortino_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #stuzer
    print(7)
    def st_f(I, mu, r_f, w):
        return -np.log(np.mean(np.exp(I * w @ (mu - r_f))))
    res = minimize(lambda I: -st_f(I, mu, r_f, w), x0=1.0, method='BFGS')
    I_st = res.x[0]
    portfolio_stuzer_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) * np.sqrt(2 * abs(I_st)) )
    
    #modiliani
    print(8)
    portfolio_modiliani_per.append((portfolio[0] - r_f) * (np.var(msci_y['MSCI']) / np.var(portfolio_return_per)) + r_f)
    
    #var
    print(9)
    alpha=0.95
    days=30
    var = abs(portfolio[0] * days - abs(norm.ppf(1-alpha)) * portfolio[1] * np.sqrt(days)) / 100
    portfolio_var_per.append(portfolio_wealth_per[-1] * var)
    
    #cvar
    print(10)
    alpha=0.95
    portfolio_ret = pdata_pct @ w
    var_level = int((1 - alpha) * len(mu))
    sorted_ret = np.sort(portfolio_ret)
    var = -sorted_ret[var_level]
    cvar = -np.mean(sorted_ret[:var_level])
    portfolio_cvar_per.append(portfolio_wealth_per[-1] * cvar)
    
    #r/cvar
    print(11)
    pseudo_ret, pseudo_cvar = get_r_cvar_portfolio(pdata
                                                   , predicted_days=30
                                                   , n_sim=1000
                                                   , mu=None
                                                   , sigma=None
                                                   , corr_matrix=None
                                                   , weight=w)
    portfolio_r_cvar_per.append(pseudo_ret / -pseudo_cvar)
    
    #alpha
    print(12)
    r_m = (msci_y.iloc[0, 0] - msci_y.iloc[-1, 0]) / msci_y.iloc[-1, 0]
    if i == 0:
        b_p = 0
    else:
        msci_y_pct = msci_y.pct_change().dropna()['MSCI']
        if len(portfolio_return_per) <= len(msci_y_pct):
            x = len(portfolio_return_per)
            b_p = np.cov(portfolio_return_per, msci_y_pct[:x])[0][0] / np.var(msci_y_pct)
        else:
            x = len(msci_y_pct)
            b_p = np.cov(portfolio_return_per[-x:], msci_y_pct)[0][0] / np.var(msci_y_pct)
        
    portfolio_alpha_per.append(portfolio[0] - (r_f + (r_m - r_f) * b_p))
    
    clear_output(wait=True)
    
all_results.append([portfolio_return_itg
                    , portfolio_return_per
                    , portfolio_wealth_per
                    , portfolio_cov_per
                    , portfolio_sharpe_per # / 10
                    , portfolio_treynor_per # / 1000000
                    , portfolio_sortino_per # / 100
                    , portfolio_stuzer_per # / 10000
                    , portfolio_modiliani_per # / 10000000
                    , portfolio_var_per
                    , portfolio_cvar_per 
                    , portfolio_r_cvar_per # / 100
                    , portfolio_alpha_per
                    , portfolio_weights])

CPU times: total: 1h 23min 7s
Wall time: 20min 53s


In [28]:
%%time

# 8 portfolio

portfolio_return_itg = [] #
portfolio_return_per = [] #
portfolio_wealth_per = [] #
portfolio_cov_per = [] #

portfolio_sharpe_per = [] #
portfolio_treynor_per = [] #
portfolio_sortino_per = [] #
portfolio_stuzer_per = [] #
portfolio_modiliani_per = [] #
portfolio_var_per = [] #
portfolio_cvar_per = [] #
portfolio_r_cvar_per = [] #
portfolio_alpha_per = [] #

portfolio_weights = [] #

for i in range(61):
    pdata = period_data[i]
    pdata_pct = period_data_pct[i]
    pdata_pct_m = period_data_pct_m[i]
    r_f = period_rate[i]
    msci_y = period_msci[i]
    
    print(i)
    print('start')
    mu = pdata_pct_m.mean()
    cov_m = pdata_pct_m.cov()
    _, portfolio, _, _, _, rcvar = get_max_r_cvar_portfolio(pdata, p=i)
    
    #return, cov, weights
    print(1)
    portfolio_return_per.append(portfolio[0])
    portfolio_cov_per.append(portfolio[1])
    portfolio_weights.append(portfolio[2])
    w = np.array(portfolio[2])
    
    #wealth periodic
    print(2)
    if i == 0:
        quant = (w * 1000000 /  pdata.iloc[-1].values).astype(int)
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
    else:
        portfolio_wealth_per.append(sum(quant * pdata.iloc[-1].values))
        quant = (w * portfolio_wealth_per[-1] /  pdata.iloc[-1].values).astype(int) 
    
    #itg return
    print(3)
    if i == 60:
        portfolio_return_itg.append(portfolio_wealth_per[-1] / portfolio_wealth_per[0])
        
    #sharpe
    print(4)
    portfolio_sharpe_per.append( (w @ (mu.values - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #treynor
    print(5)
    betas = get_asset_betas(pdata_pct_m, msci_y['MSCI'])
    portfolio_treynor_per.append( (mu - r_f) @ w / (w @ betas.T) )
    
    #sortino
    print(6)
    portfolio_sortino_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) )
    
    #stuzer
    print(7)
    def st_f(I, mu, r_f, w):
        return -np.log(np.mean(np.exp(I * w @ (mu - r_f))))
    res = minimize(lambda I: -st_f(I, mu, r_f, w), x0=1.0, method='BFGS')
    I_st = res.x[0]
    portfolio_stuzer_per.append( (w @ (mu - r_f)) / np.sqrt(w @ cov_m @ w) * np.sqrt(2 * abs(I_st)) )
    
    #modiliani
    print(8)
    portfolio_modiliani_per.append((portfolio[0] - r_f) * (np.var(msci_y['MSCI']) / np.var(portfolio_return_per)) + r_f)
    
    #var
    print(9)
    alpha=0.95
    days=30
    var = abs(portfolio[0] * days - abs(norm.ppf(1-alpha)) * portfolio[1] * np.sqrt(days)) / 100
    portfolio_var_per.append(portfolio_wealth_per[-1] * var)
    
    #cvar
    print(10)
    alpha=0.95
    portfolio_ret = pdata_pct @ w
    var_level = int((1 - alpha) * len(mu))
    sorted_ret = np.sort(portfolio_ret)
    var = -sorted_ret[var_level]
    cvar = -np.mean(sorted_ret[:var_level])
    portfolio_cvar_per.append(portfolio_wealth_per[-1] * cvar)
    
    #r/cvar
    print(11)
    #pseudo_ret, pseudo_cvar = get_r_cvar_portfolio(pdata
    #                                               , predicted_days=30
    #                                               , n_sim=1000
    #                                               , mu=None
    #                                               , sigma=None
    #                                               , corr_matrix=None
    #                                               , weight=w)
    portfolio_r_cvar_per.append(rcvar)
    
    #alpha
    print(12)
    r_m = (msci_y.iloc[0, 0] - msci_y.iloc[-1, 0]) / msci_y.iloc[-1, 0]
    if i == 0:
        b_p = 0
    else:
        msci_y_pct = msci_y.pct_change().dropna()['MSCI']
        if len(portfolio_return_per) <= len(msci_y_pct):
            x = len(portfolio_return_per)
            b_p = np.cov(portfolio_return_per, msci_y_pct[:x])[0][0] / np.var(msci_y_pct)
        else:
            x = len(msci_y_pct)
            b_p = np.cov(portfolio_return_per[-x:], msci_y_pct)[0][0] / np.var(msci_y_pct)
        
    portfolio_alpha_per.append(portfolio[0] - (r_f + (r_m - r_f) * b_p))
    
    clear_output(wait=True)
    
all_results.append([portfolio_return_itg
                    , portfolio_return_per
                    , portfolio_wealth_per
                    , portfolio_cov_per
                    , portfolio_sharpe_per # 
                    , portfolio_treynor_per # / 10000
                    , portfolio_sortino_per # 
                    , portfolio_stuzer_per # / 100
                    , portfolio_modiliani_per # / 10000000
                    , portfolio_var_per
                    , portfolio_cvar_per 
                    , portfolio_r_cvar_per # 
                    , portfolio_alpha_per
                    , portfolio_weights])

CPU times: total: 23h 42min 3s
Wall time: 14h 51min 32s


In [44]:
all_results_mod = []

for kk, port in enumerate(all_results):
    p_ms = []
    for gg, metr in enumerate(port):
        if kk == 0:
            if gg == 4:
                metr = np.array(metr)
                metr = metr / 100
            if gg == 5:
                metr = np.array(metr)
                metr = metr / 1000000
            if gg == 6:
                metr = np.array(metr)
                metr = metr / 100
            if gg == 7:
                metr = np.array(metr)
                metr = metr / 10000
            if gg == 8:
                metr = np.array(metr)
                metr = metr / 10000000
            if gg == 11:
                metr = np.array(metr)
                metr = metr / 100
        if kk == 1:
            if gg == 4:
                metr = np.array(metr)
                metr = metr / 1000
            if gg == 5:
                metr = np.array(metr)
                metr = metr / 1000000
            if gg == 6:
                metr = np.array(metr)
                metr = metr / 100
            if gg == 7:
                metr = np.array(metr)
                metr = metr / 100000
            if gg == 8:
                metr = np.array(metr)
                metr = metr / 10000000
            if gg == 11:
                metr = np.array(metr)
                metr = metr / 100
        if kk == 2:
            if gg == 4:
                metr = np.array(metr)
                metr = metr / 10
            if gg == 5:
                metr = np.array(metr)
                metr = metr / 10000000
            if gg == 6:
                metr = np.array(metr)
                metr = metr / 10
            if gg == 7:
                metr = np.array(metr)
                metr = metr / 10000
            if gg == 8:
                metr = np.array(metr)
                metr = metr / 10000000
            if gg == 9:
                metr = np.array(metr)
                metr = metr / 10
            if gg == 11:
                metr = np.array(metr)
                metr = metr / 100
        if kk == 3:
            if gg == 4:
                metr = np.array(metr)
                metr = metr / 10
            if gg == 5:
                metr = np.array(metr)
                metr = metr / 1000
            if gg == 6:
                metr = np.array(metr)
                metr = metr / 10
            if gg == 7:
                metr = np.array(metr)
                metr = metr / 100
            if gg == 8:
                metr = np.array(metr)
                metr = metr / 10000000
        if kk == 4:
            if gg == 5:
                metr = np.array(metr)
                metr = metr / 100000
            if gg == 7:
                metr = np.array(metr)
                metr = metr / 100
            if gg == 8:
                metr = np.array(metr)
                metr = metr / 10000000000
            if gg == 9:
                metr = np.array(metr)
                metr = metr * 10
        if kk == 5:
            if gg == 4:
                metr = np.array(metr)
                metr = metr / 10
            if gg == 5:
                metr = np.array(metr)
                metr = metr / 1000000
            if gg == 6:
                metr = np.array(metr)
                metr = metr / 100
            if gg == 7:
                metr = np.array(metr)
                metr = metr / 10000
            if gg == 8:
                metr = np.array(metr)
                metr = metr / 10000000
            if gg == 11:
                metr = np.array(metr)
                metr = metr / 100
        if kk == 6:
            if gg == 5:
                metr = np.array(metr)
                metr = metr / 10000
            if gg == 7:
                metr = np.array(metr)
                metr = metr / 100
            if gg == 8:
                metr = np.array(metr)
                metr = metr / 10000000
            if gg == 11:
                #metr = np.array(metr)
                #metr = [np.mean(i) for i in metr]
                pre_metr = []
                for ii in metr:
                    pre_metr.append(np.mean([iii[0] / iii[1] for iii in ii]))
                metr = np.array(pre_metr)
                
        p_ms.append(metr)
    all_results_mod.append(p_ms)

In [34]:
from IPython.display import clear_output
import time

In [47]:
import pickle

with open('portf_all.pkl', 'wb') as f:
    pickle.dump(all_results_mod, f)